In [1]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
import argparse
from datetime import datetime
import pytz
import csv

c:\Users\Tardy\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
def get_scale(enc):
    s = enc.layers[0].output_shape[0][1]
    s = 1024//s
    return s

In [3]:
def change_model(dec,s):
    inp_shape = (32//s,32//s,8)
    code_input_layer = keras.Input(shape=inp_shape, name="CODE_INPUT")
    x = code_input_layer
    for lay in dec.layers[1:]:
        x = lay(x)
    new_dec = keras.Model(code_input_layer,x)
    return new_dec

In [17]:
fld = f"./autoencoder_models/2022-08-22-11-06"
encoder = keras.models.load_model(f"{fld}/encoder.h5", compile=False)
decoder = keras.models.load_model(f"{fld}/decoder.h5", compile=False)
encoder.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 INPUT (InputLayer)          [(None, 256, 256, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 256, 256, 16)      800       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 8)       10376     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 64, 64, 8)        0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 8)         7752

In [13]:
s = get_scale(encoder)
d = change_model(decoder,s)
d.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 CODE_INPUT (InputLayer)     [(None, 8, 8, 8)]         0         
                                                                 
 conv2d_transpose (Conv2DTra  multiple                 3144      
 nspose)                                                         
                                                                 
 up_sampling2d (UpSampling2D  multiple                 0         
 )                                                               
                                                                 
 conv2d_transpose_1 (Conv2DT  multiple                 5192      
 ranspose)                                                       
                                                                 
 up_sampling2d_1 (UpSampling  multiple                 0         
 2D)                                                       